In [ ]:
import random
import datetime
import hashlib
from collections import Counter
from math import sqrt
from functools import lru_cache
from collections import deque, defaultdict

class Cell:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.ent = None

    def empty(self):
        return self.ent is None

class Entity:
    def __init__(self, color):
        self.color = color

class Eater(Entity): pass

class Food(Entity):
    def __init__(self, color, is_last=False):
        super().__init__(color)
        self.is_last = is_last


class Grid:
    DIRS = [(0,1), (0,-1), (1,0), (-1,0)]
    COLOR_MAP = {
        'Food': {'White': 0, 'Green': 1, 'Blue': 2, 'Red': 3, 'Yellow': 4, 'Purple': 5, 'Pink': 6, 'Brown': 7},
        'Eater': {'Green': 0, 'Blue': 1, 'Red': 2, 'Yellow': 3, 'Purple': 4, 'Pink': 5, 'Brown': 6}
    }
    
    # Default scoring weights
    DEFAULT_WEIGHTS = {
        'food_separation': 5.0,
        'cluster_size': 0.33,
        'clusters_count': 1.0,
        'distribution': 1.0,
        'distance_to_eater': 1.0,
        'path_complexity': 0.8,
        'interaction_complexity': 1.5,
        'empty_cells': 2.0,
        'solution_complexity': 2.0,  # Weight for solution complexity
        'move_interdependency': 1.5,  # Weight for move dependencies
        'critical_path': 1.2,        # Weight for critical path score
    }

    def __init__(self, rows, cols, weights=None):
        self.rows = rows
        self.cols = cols
        self.grid = [[Cell(x, y) for y in range(cols)] for x in range(rows)]
        self.moves = []
        self.min_separation = 2
        self._distance_cache = {}
        self._cluster_cache = {}
        self._path_cache = {}
        self._food_by_color = defaultdict(list)
        self._eater_positions = {}
        # Initialize weights with defaults or custom values
        self.weights = weights if weights is not None else self.DEFAULT_WEIGHTS.copy()

    def get_board_signature(self):
        """Generate a unique signature for the board layout to detect similar boards"""
        # Create a representation of the board including entities and their positions
        board_repr = []
        for x in range(self.rows):
            for y in range(self.cols):
                cell = self.grid[x][y]
                if cell.ent is None:
                    board_repr.append(f"E_{x}_{y}")  # Empty
                elif isinstance(cell.ent, Food):
                    board_repr.append(f"F_{cell.ent.color}_{x}_{y}")
                elif isinstance(cell.ent, Eater):
                    board_repr.append(f"E_{cell.ent.color}_{x}_{y}")
        
        # Sort to ensure consistent ordering regardless of traversal
        board_repr.sort()
        
        # Create a hash of the representation
        signature = hashlib.md5(str(board_repr).encode()).hexdigest()
        return signature

    def calculate_food_balance_score(self):
        """Calculate how balanced the food is distributed among eaters after whitening"""
        # Update food cache 
        self._update_entity_cache()
        
        # Count colored food items for each eater
        food_counts = {color: len(foods) for color, foods in self._food_by_color.items() 
                      if color != 'White' and color in self._eater_positions}
        
        if not food_counts:
            return 0
        
        # Calculate statistics
        avg_food = sum(food_counts.values()) / len(food_counts)
        max_deviation = max(abs(count - avg_food) for count in food_counts.values())
        
        # Normalize to a 0-100 score (lower deviation = higher score)
        max_possible_deviation = max(len(self._food_by_color.get(color, [])) for color in food_counts)
        if max_possible_deviation == 0:
            return 100  # Perfect balance (no food)
        
        balance_score = 100 * (1 - (max_deviation / max_possible_deviation))
        return balance_score
        
    def calculate_clustering_score(self, minimum_qualified_colors):
        colors = set(self.COLOR_MAP['Food'].keys()) - {'White'}
        metrics = {}
        
        # Update food locations cache
        self._update_entity_cache()
        
        for color in colors:
            foods = self._food_by_color.get(color, [])
            
            if not foods:
                continue

            eater_pos = self._eater_positions.get(color)
            
            if not eater_pos:
                continue

            # Use the cached position tuples
            foods_tuple = tuple(sorted(foods))
            
            color_metrics = {
                'food_count': len(foods),
                'avg_food_separation': self._calculate_avg_separation(foods_tuple),
                'largest_cluster_size': self._find_largest_cluster(foods_tuple),
                'clusters_count': self._count_clusters(foods_tuple),
                'avg_distance_to_eater': self._calculate_avg_distance_to_point(foods, eater_pos),
                'distribution_score': self._calculate_distribution_score(foods),
                'path_complexity': self._calculate_path_complexity(color)
            }
            
            if (color_metrics['largest_cluster_size'] == color_metrics['food_count'] and 
                color_metrics['food_count'] > 1):
                color_metrics['disqualified'] = True
            else:
                color_metrics['disqualified'] = False
                
            metrics[color] = color_metrics

        if not metrics:
            return {'final_score': float('inf'), 'color_metrics': {}, 'disqualified': True}

        disqualified = sum(0 if m['disqualified'] else 1 for m in metrics.values()) < minimum_qualified_colors
        
        # Calculate interaction complexity between different color eaters
        interaction_complexity = self._calculate_eater_interactions()
        
        # Calculate empty cells score
        empty_cells = self.count_empty()
        empty_cells_score = self._calculate_empty_cells_score(empty_cells)
        solution_complexity = self._calculate_solution_complexity()
        move_interdependency = self._calculate_move_interdependency()
        critical_path_score = self._calculate_critical_path()
        
        final_score = float('inf') if disqualified else self._calculate_final_score(
            metrics, interaction_complexity, empty_cells_score, 
            solution_complexity, move_interdependency, critical_path_score)
        
        return {
            'final_score': final_score,
            'color_metrics': metrics,
            'disqualified': disqualified,
            'interaction_complexity': interaction_complexity,
            'empty_cells': empty_cells,
            'empty_cells_score': empty_cells_score,
            'solution_complexity': solution_complexity,
            'move_interdependency': move_interdependency,
            'critical_path_score': critical_path_score
        }
    
    def _calculate_solution_complexity(self):
        """Calculate complexity score based on the solution path"""
        if not self.moves:
            return 0
        
        # Group moves by eater color
        moves_by_eater = {}
        for move in self.moves:
            color = move['eater_color']
            if color not in moves_by_eater:
                moves_by_eater[color] = []
            moves_by_eater[color].append(move)
        
        complexity_score = 0
        
        for color, eater_moves in moves_by_eater.items():
            # Score based on move patterns
            direction_changes = 0
            backtracking_moves = 0
            last_direction = None
            visited_positions = set()
            
            for i, move in enumerate(eater_moves):
                start, end = move['start'], move['end']
                
                # Track direction changes
                dx = end[0] - start[0]
                dy = end[1] - start[1]
                
                current_direction = None
                if dx != 0 and dy == 0:
                    current_direction = "horizontal"
                elif dx == 0 and dy != 0:
                    current_direction = "vertical"
                
                if last_direction and current_direction and last_direction != current_direction:
                    direction_changes += 1
                
                last_direction = current_direction
                
                # Track backtracking (revisiting cells)
                if start in visited_positions:
                    backtracking_moves += 1
                
                visited_positions.add(start)
                visited_positions.add(end)
                
                # Add distance as a factor
                complexity_score += move['dist'] * 0.5
            
            # Add scores from pattern analysis
            complexity_score += direction_changes * 2.0
            complexity_score += backtracking_moves * 3.0
            
            # Bonus for longer solution sequences
            complexity_score += len(eater_moves) * 0.5
        
        return complexity_score
    
    def _calculate_move_interdependency(self):
        """Calculate how much moves depend on each other"""
        if not self.moves:
            return 0
        
        # Create a dependency graph of moves
        dependency_score = 0
        food_positions = {}
        eater_positions = {}
        
        # Sort moves by distance (potentially optimal order)
        sorted_moves = sorted(self.moves, key=lambda m: m['dist'])
        
        for move in sorted_moves:
            color = move['eater_color']
            start, end = move['start'], move['end']
            
            # Track positions
            if color not in eater_positions:
                eater_positions[color] = start
            else:
                # Check if this move depends on previous moves
                current_pos = eater_positions[color]
                if current_pos != start:
                    # This move requires the eater to be in a specific position
                    dependency_score += 2
            
            # Update position after move
            eater_positions[color] = end
            
            # Check if this move potentially blocks other eaters
            block_score = 0
            for other_color, other_pos in eater_positions.items():
                if other_color != color:
                    # Check if the food placed blocks a path
                    if self._is_potentially_blocking(end, other_pos, food_positions.get(other_color, [])):
                        block_score += 3
            
            dependency_score += block_score
            
            # Track food positions
            if color not in food_positions:
                food_positions[color] = []
            food_positions[color].append(end)
        
        return dependency_score
    
    def _is_potentially_blocking(self, food_pos, eater_pos, other_foods):
        """Check if a food position potentially blocks an eater's path"""
        if not other_foods:
            return False
        
        # Check if the food is on a direct path between eater and any of its foods
        for other_food in other_foods:
            # Check if food is on the rectangle formed by eater and its food
            min_x = min(eater_pos[0], other_food[0])
            max_x = max(eater_pos[0], other_food[0])
            min_y = min(eater_pos[1], other_food[1])
            max_y = max(eater_pos[1], other_food[1])
            
            if min_x <= food_pos[0] <= max_x and min_y <= food_pos[1] <= max_y:
                # Check if it's directly on the path (Manhattan distance)
                if (abs(eater_pos[0] - food_pos[0]) + abs(eater_pos[1] - food_pos[1]) + 
                    abs(food_pos[0] - other_food[0]) + abs(food_pos[1] - other_food[1])) == \
                   (abs(eater_pos[0] - other_food[0]) + abs(eater_pos[1] - other_food[1])):
                    return True
        
        return False
    
    def _calculate_critical_path(self):
        """Calculate critical path score - identifying bottleneck moves"""
        if not self.moves:
            return 0
        
        # Group moves by eater color
        moves_by_eater = {}
        for move in self.moves:
            color = move['eater_color']
            if color not in moves_by_eater:
                moves_by_eater[color] = []
            moves_by_eater[color].append(move)
        
        critical_score = 0
        visited_positions = set()  # Track all positions used by any eater
        choke_points = set()       # Track positions used by multiple eaters
        
        # First pass: Identify all positions used in moves
        for moves in moves_by_eater.values():
            for move in moves:
                start_pos = move['start']
                end_pos = move['end']
                
                # Check if these positions have been used before
                if start_pos in visited_positions:
                    choke_points.add(start_pos)
                if end_pos in visited_positions:
                    choke_points.add(end_pos)
                
                # Add to visited
                visited_positions.add(start_pos)
                visited_positions.add(end_pos)
        
        # Add score for choke points (positions used by multiple eaters)
        critical_score += len(choke_points) * 3
        
        # Analyze each eater's moves to find critical paths
        for color, eater_moves in moves_by_eater.items():
            # Identify forced move sequences (moves that must be done in order)
            current_sequence_length = 1  # Start with 1 (first move)
            max_sequence_length = 1
            
            # Check for sequences where moves must follow each other
            for i in range(len(eater_moves) - 1):
                current_move = eater_moves[i]
                next_move = eater_moves[i + 1]
                
                # If the end of current move is the start of next move, it's a forced sequence
                if current_move['end'] == next_move['start']:
                    current_sequence_length += 1
                    max_sequence_length = max(max_sequence_length, current_sequence_length)
                else:
                    # Sequence broken
                    current_sequence_length = 1
            
            # Add score based on longest forced sequence (exponential scoring)
            if max_sequence_length > 1:
                critical_score += max_sequence_length * 2
            
            # Check for moves that go through choke points
            choke_point_moves = 0
            for move in eater_moves:
                if move['start'] in choke_points or move['end'] in choke_points:
                    choke_point_moves += 1
            
            # Add score for moves through choke points
            critical_score += choke_point_moves * 2
        
        # Add bonus for boards with many eaters (more potential for critical paths)
        critical_score += len(moves_by_eater) * 0.5
        
        return critical_score
    
    def _calculate_empty_cells_score(self, empty_cells):
        """Calculate a score based on how few empty cells are in the board.
        Fewer empty cells = higher score."""
        total_cells = self.rows * self.cols
        filled_ratio = 1 - (empty_cells / total_cells)
        # Higher score for more filled boards (fewer empty cells)
        return filled_ratio * 100
    
    def _update_entity_cache(self):
        """Update cached positions of foods by color and eaters"""
        self._food_by_color.clear()
        self._eater_positions.clear()
        
        for x in range(self.rows):
            for y in range(self.cols):
                cell = self.grid[x][y]
                if isinstance(cell.ent, Food):
                    self._food_by_color[cell.ent.color].append((x, y))
                elif isinstance(cell.ent, Eater):
                    self._eater_positions[cell.ent.color] = (x, y)

    @lru_cache(maxsize=4096)
    def _manhattan_distance(self, pos1, pos2):
        return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])

    def _calculate_avg_separation(self, positions):
        if len(positions) < 2:
            return float('inf')
        
        cache_key = positions
        if cache_key in self._distance_cache:
            return self._distance_cache[cache_key]
            
        distances = []
        for i, pos1 in enumerate(positions[:-1]):
            for pos2 in positions[i+1:]:
                distances.append(self._manhattan_distance(pos1, pos2))
        
        result = sum(distances) / len(distances) if distances else float('inf')
        self._distance_cache[cache_key] = result
        return result

    def _count_clusters(self, positions, cluster_threshold=2):
        if not positions:
            return 0
            
        cache_key = (positions, cluster_threshold)
        if cache_key in self._cluster_cache:
            return self._cluster_cache[cache_key]
            
        positions_set = set(positions)
        clusters = 0
        
        while positions_set:
            clusters += 1
            start = next(iter(positions_set))
            positions_set.remove(start)
            
            queue = deque([start])
            while queue:
                current = queue.popleft()
                for pos in list(positions_set):
                    if self._manhattan_distance(current, pos) <= cluster_threshold:
                        queue.append(pos)
                        positions_set.remove(pos)
        
        self._cluster_cache[cache_key] = clusters
        return clusters

    def _find_largest_cluster(self, positions, cluster_threshold=2):
        if not positions:
            return 0
            
        cache_key = (positions, cluster_threshold)
        if cache_key in self._cluster_cache:
            return self._cluster_cache.get(f"largest_{cache_key}", 0)
            
        positions_set = set(positions)
        largest_cluster = 0
        
        while positions_set:
            start = next(iter(positions_set))
            positions_set.remove(start)
            
            queue = deque([start])
            cluster_size = 1
            
            while queue:
                current = queue.popleft()
                for pos in list(positions_set):
                    if self._manhattan_distance(current, pos) <= cluster_threshold:
                        queue.append(pos)
                        positions_set.remove(pos)
                        cluster_size += 1
            
            largest_cluster = max(largest_cluster, cluster_size)
        
        self._cluster_cache[f"largest_{cache_key}"] = largest_cluster
        return largest_cluster

    def _calculate_avg_distance_to_point(self, positions, target):
        if not positions:
            return 0
        distances = [self._manhattan_distance(pos, target) for pos in positions]
        return sum(distances) / len(distances)

    def _calculate_distribution_score(self, positions):
        if not positions:
            return 0
            
        mid_row = self.rows // 2
        mid_col = self.cols // 2
        quadrants = [0] * 4
        
        for x, y in positions:
            quad_idx = (x >= mid_row) * 2 + (y >= mid_col)
            quadrants[quad_idx] += 1
            
        avg = sum(quadrants) / 4
        variance = sum((q - avg) ** 2 for q in quadrants) / 4
        return sqrt(variance)
    
    def _calculate_path_complexity(self, color):
        """Calculate the complexity of the path an eater must take to collect its food"""
        eater_pos = self._eater_positions.get(color)
        if not eater_pos:
            return 0
            
        foods = self._food_by_color.get(color, [])
        if not foods:
            return 0
            
        # Cache key for this specific configuration
        cache_key = (eater_pos, tuple(sorted(foods)))
        if cache_key in self._path_cache:
            return self._path_cache[cache_key]
            
        # Find optimal path
        current_pos = eater_pos
        remaining_foods = set(foods)
        total_distance = 0
        direction_changes = 0
        prev_direction = None
        
        while remaining_foods:
            # Find nearest food
            nearest_food = min(remaining_foods, key=lambda pos: self._manhattan_distance(current_pos, pos))
            remaining_foods.remove(nearest_food)
            
            # Calculate path metrics
            dx = nearest_food[0] - current_pos[0]
            dy = nearest_food[1] - current_pos[1]
            
            # Calculate direction changes
            if dx != 0 and dy != 0:
                # Both directions need change
                direction_changes += 1
                
            current_direction = None
            if abs(dx) > abs(dy):
                current_direction = "horizontal"
            elif abs(dy) > abs(dx):
                current_direction = "vertical"
                
            if prev_direction and current_direction and prev_direction != current_direction:
                direction_changes += 1
                
            prev_direction = current_direction
            
            # Update position and add distance
            total_distance += abs(dx) + abs(dy)
            current_pos = nearest_food
            
        complexity = total_distance * 0.5 + direction_changes * 2.0
        self._path_cache[cache_key] = complexity
        return complexity
    
    def _calculate_eater_interactions(self):
        """Calculate the complexity of interactions between eaters"""
        eaters = list(self._eater_positions.items())
        if len(eaters) < 2:
            return 0
            
        interaction_score = 0
        
        # Calculate space contention between eaters
        for i, (color1, pos1) in enumerate(eaters[:-1]):
            for color2, pos2 in eaters[i+1:]:
                # Eaters that are close might contend for space
                distance = self._manhattan_distance(pos1, pos2)
                if distance < self.rows + self.cols / 3:
                    interaction_score += (self.rows + self.cols) / (distance + 1)
                    
                # Check for potential path crossing
                foods1 = set(self._food_by_color.get(color1, []))
                foods2 = set(self._food_by_color.get(color2, []))
                
                # Check if eaters need to cross paths to reach their foods
                for f1 in foods1:
                    for f2 in foods2:
                        # If paths might intersect
                        if (self._manhattan_distance(pos1, f2) + self._manhattan_distance(pos2, f1) <
                            self._manhattan_distance(pos1, f1) + self._manhattan_distance(pos2, f2)):
                            interaction_score += 5
        
        # Bonus for eaters that are surrounded by other colors' food
        for color, pos in eaters:
            other_foods = []
            for c, foods in self._food_by_color.items():
                if c != color and c != 'White':
                    other_foods.extend(foods)
            
            # Count nearby foods of other colors
            nearby_other_foods = sum(1 for f in other_foods if self._manhattan_distance(pos, f) < 3)
            interaction_score += nearby_other_foods * 2
                    
        return interaction_score

    def _calculate_final_score(self, metrics, interaction_complexity, empty_cells_score, 
                              solution_complexity, move_interdependency, critical_path_score):
        """Calculate final score including solution-based metrics and food balance"""
        # Original score calculation...
        scores = []
        for color_metrics in metrics.values():
            if color_metrics.get('disqualified', False):
                continue
                
            color_score = (
                (self.weights['food_separation'] / (color_metrics['avg_food_separation'] + 0.1)) +
                (color_metrics['largest_cluster_size'] * self.weights['cluster_size']) +
                (self.weights['clusters_count'] / (color_metrics['clusters_count'] + 0.1)) +
                (color_metrics['distribution_score'] * self.weights['distribution']) +
                (self.weights['distance_to_eater'] / (color_metrics['avg_distance_to_eater'] + 1)) +
                (color_metrics['path_complexity'] * self.weights['path_complexity'])
            )
            scores.append(color_score)
        
        base_score = sum(scores) / len(scores) if scores else float('inf')
        
        # Calculate food balance score after whitening
        food_balance_score = self.calculate_food_balance_score()
        
        # Add food balance weight to weights if not present
        if 'food_balance' not in self.weights:
            self.weights['food_balance'] = 1.8  # Default weight for food balance
        
        # Add solution-based metrics to the final score
        return (base_score + 
               (interaction_complexity * self.weights['interaction_complexity']) +
               (empty_cells_score * self.weights['empty_cells']) +
               (solution_complexity * self.weights['solution_complexity']) +
               (move_interdependency * self.weights['move_interdependency']) +
               (critical_path_score * self.weights['critical_path']) +
               (food_balance_score * self.weights['food_balance']))
    
    # Method to update weights
    def set_weights(self, new_weights):
        """Update the scoring weights with new values"""
        for key, value in new_weights.items():
            if key in self.weights:
                self.weights[key] = value
    
    def place(self, ent, x, y):
        self.grid[x][y].ent = ent
        # Update entity cache after placement
        if isinstance(ent, Food):
            self._food_by_color[ent.color].append((x, y))
        elif isinstance(ent, Eater):
            self._eater_positions[ent.color] = (x, y)

    def clear(self):
        for row in self.grid:
            for cell in row: cell.ent = None
        # Clear caches
        self._distance_cache.clear()
        self._cluster_cache.clear()
        self._path_cache.clear()
        self._food_by_color.clear()
        self._eater_positions.clear()

    def gen_puzzle(self, eaters, white_pct, min_food):
        self.clear()
        eater_pos = []
        
        colors = [c for c, n in eaters.items() for _ in range(n)]
        random.shuffle(colors)
        
        # Place eaters first
        for color in colors:
            pos = self.get_empty()
            if not pos: break
            x, y = pos
            e = Eater(color)
            self.place(e, x, y)
            eater_pos.append({'e': e, 'x': x, 'y': y, 'count': 0})

        # First pass: ensure minimum food count
        active_eaters = eater_pos.copy()
        while active_eaters:
            moved_any = False
            # Shuffle eaters each round to create more interesting interactions
            random.shuffle(active_eaters)
            
            for einfo in active_eaters[:]:
                if einfo['count'] >= min_food:
                    active_eaters.remove(einfo)
                    continue
                    
                # Try to move using strategic placement
                if self.move_eater_strategically(einfo, eater_pos):
                    moved_any = True
                else:
                    active_eaters.remove(einfo)
            
            if not moved_any:
                break

        # Second pass: fill remaining spaces where possible
        remaining_spaces = self.count_empty()
        active_eaters = [e for e in eater_pos if self.can_move_eater(e)]
        random.shuffle(active_eaters)
        
        failures = 0
        max_failures = 10
        
        while remaining_spaces > 0 and active_eaters and failures < max_failures:
            moved_any = False
            
            for einfo in active_eaters[:]:
                if self.move_eater_strategically(einfo, eater_pos):
                    moved_any = True
                    remaining_spaces -= 1
                    failures = 0
                else:
                    active_eaters.remove(einfo)
                    
                if remaining_spaces <= 0:
                    break
            
            if not moved_any:
                failures += 1
            
            active_eaters = [e for e in eater_pos if self.can_move_eater(e)]
            random.shuffle(active_eaters)

        self.whiten_food(white_pct)

    def can_move_eater(self, einfo):
        for dx, dy in self.DIRS:
            x, y = einfo['x'], einfo['y']
            new_x, new_y = self.get_max_pos(x, y, dx, dy)
            if new_x is not None and (new_x != x or new_y != y):
                return True
        return False

    def move_eater_strategically(self, einfo, all_eaters):
        """Move eater with improved strategic decision making for puzzle complexity"""
        random.shuffle(self.DIRS)
        best_move = None
        best_score = float('-inf')
        
        # Consider other eaters' positions for interaction complexity
        other_eaters = [(e['x'], e['y']) for e in all_eaters if e['e'].color != einfo['e'].color]
        
        for dx, dy in self.DIRS:
            x, y = einfo['x'], einfo['y']
            new_x, new_y = self.get_max_pos(x, y, dx, dy)
            
            if new_x is None or (new_x == x and new_y == y):
                continue
            
            old_ent = self.grid[x][y].ent
            self.grid[x][y].ent = Food(einfo['e'].color)
            
            # Evaluate food placement with consideration of other eaters
            move_score = self._evaluate_food_placement(x, y, einfo['e'].color)
            
            # Add bonus for moves that create interesting eater interactions
            for ex, ey in other_eaters:
                # Path blocking score - higher if this food might block another eater's path
                distance_to_other = self._manhattan_distance((x, y), (ex, ey))
                if distance_to_other < 3:
                    move_score += 20
                elif distance_to_other < 5:
                    move_score += 10
            
            self.grid[x][y].ent = old_ent
            
            if move_score > best_score:
                best_score = move_score
                best_move = (new_x, new_y)
        
        # Fallback to any valid move if no good strategic move found
        if best_move is None:
            for dx, dy in self.DIRS:
                x, y = einfo['x'], einfo['y']
                new_x, new_y = self.get_max_pos(x, y, dx, dy)
                if new_x is not None and (new_x != x or new_y != y):
                    best_move = (new_x, new_y)
                    break
        
        if best_move:
            new_x, new_y = best_move
            x, y = einfo['x'], einfo['y']
            
            is_first_food = einfo['count'] == 0
            
            self.grid[x][y].ent = None
            self.place(einfo['e'], new_x, new_y)
            self.place(Food(einfo['e'].color, is_last=is_first_food), x, y)
            einfo['count'] += abs(new_x - x) + abs(new_y - y)
            
            self.moves.append({
                'eater_color': einfo['e'].color,
                'start': (new_x, new_y),
                'end': (x, y),
                'food_color': einfo['e'].color,
                'dist': abs(new_x - x) + abs(new_y - y)
            })
            
            einfo['x'], einfo['y'] = new_x, new_y
            
            # Clear path cache as it's now invalid
            self._path_cache.clear()
            return True
            
        return False
    
    def _evaluate_food_placement(self, x, y, color):
        """Evaluate how good a food placement is with improved metrics"""
        # Use the cached food positions
        same_color_foods = self._food_by_color.get(color, [])
        
        if not same_color_foods:
            return 100
        
        min_distance = min(self._manhattan_distance((x, y), (fx, fy)) for fx, fy in same_color_foods)
        
        temp_foods = same_color_foods + [(x, y)]
        temp_foods_tuple = tuple(sorted(temp_foods))
        
        cluster_size = self._find_largest_cluster(temp_foods_tuple)
        clusters_count = self._count_clusters(temp_foods_tuple)
        
        if cluster_size == len(temp_foods) and len(temp_foods) > 1:
            return -1000
        
        # Separation score
        separation_score = 0
        if min_distance >= self.min_separation:
            separation_score = 50 + min_distance * 10
        else:
            separation_score = 20 - (self.min_separation - min_distance) * 15
        
        # Prefer multiple clusters for complexity
        cluster_score = clusters_count * 20
        
        # Prefer food placement that creates more complex paths
        path_score = 0
        eater_pos = self._eater_positions.get(color)
        if eater_pos:
            # Prefer food that requires eater to change direction to reach
            dx1 = abs(eater_pos[0] - x)
            dy1 = abs(eater_pos[1] - y)
            if dx1 > 0 and dy1 > 0:
                path_score += 30  # Diagonal relationships require more moves
            
            # Prefer food that's moderately distant (not too close, not too far)
            dist = self._manhattan_distance(eater_pos, (x, y))
            if 3 <= dist <= 6:
                path_score += 25
            elif dist > 6:
                path_score += 15
        
        return separation_score + cluster_score + path_score

    def get_max_pos(self, x, y, dx, dy):
        nx, ny = x, y
        while self.in_bounds(nx + dx, ny + dy) and self.grid[nx + dx][ny + dy].empty():
            nx += dx
            ny += dy
        return (nx, ny) if (nx, ny) != (x, y) else (None, None)

    def whiten_food(self, pct):
        # Get all food items that can be whitened
        food = [(c.x, c.y) for row in self.grid for c in row 
                if isinstance(c.ent, Food) and c.ent.color != 'White' and not c.ent.is_last]
        
        # Count food by color before whitening
        food_by_color = defaultdict(list)
        for x, y in food:
            food_by_color[self.grid[x][y].ent.color].append((x, y))
        
        # Calculate target whitening to maintain balance
        total_to_white = int(len(food) * pct / 100)
        if not total_to_white:
            return
        
        # Determine how many to whiten from each color to maintain balance
        colors = list(food_by_color.keys())
        counts = [len(food_by_color[c]) for c in colors]
        
        # Find colors with more food to prioritize whitening from them
        avg_count = sum(counts) / len(counts) if counts else 0
        excess = {colors[i]: max(0, counts[i] - avg_count) for i in range(len(colors))}
        
        # Sort colors by excess food (descending)
        sorted_colors = sorted(excess.items(), key=lambda x: x[1], reverse=True)
        
        # Distribute whitening quota proportionally among colors with excess
        to_white_by_color = {}
        remaining = total_to_white
        
        # First pass: whiten from colors with excess
        for color, excess_amount in sorted_colors:
            if excess_amount > 0 and remaining > 0:
                # Calculate proportional whitening
                color_white_count = min(int(excess_amount), remaining)
                to_white_by_color[color] = color_white_count
                remaining -= color_white_count
        
        # Second pass: distribute remaining quota evenly
        if remaining > 0:
            per_color = max(1, remaining // len(colors))
            for color in colors:
                if color not in to_white_by_color:
                    to_white_by_color[color] = 0
                additional = min(per_color, len(food_by_color[color]) - to_white_by_color[color])
                to_white_by_color[color] += additional
                remaining -= additional
                if remaining <= 0:
                    break
        
        # Final whitening
        to_white = []
        for color, count in to_white_by_color.items():
            if count > 0:
                selected = random.sample(food_by_color[color], count)
                to_white.extend(selected)
        
        # Apply whitening
        for x, y in to_white:
            self.grid[x][y].ent.color = 'White'

    def get_empty(self):
        empty = [(x,y) for x in range(self.rows) for y in range(self.cols) 
                if self.grid[x][y].empty()]
        return random.choice(empty) if empty else None

    def in_bounds(self, x, y):
        return 0 <= x < self.rows and 0 <= y < self.cols

    def count_empty(self):
        return sum(1 for row in self.grid for cell in row if cell.empty())

    def to_scene(self, level_id, fill_empty):
        width, height = 1440, 2560
        step = max(180, width // self.cols - 24)
        
        start_x = width/2 - step*self.cols/2 + step*0.5
        start_y = height/2 - step*self.rows/2 + step*0.5 - 40

        scene = f"""[gd_scene load_steps=4 format=3 uid="{level_id}"]

[ext_resource type="Script" path="res://Levels/Level.cs" id="lvlid"]
[ext_resource type="PackedScene" uid="uid://byatslmwbvorg" path="res://Entities/Food/Food.tscn" id="foodid"]
[ext_resource type="PackedScene" uid="uid://dd570jgysudow" path="res://Entities/Eater/Eater.tscn" id="eaterid"]

[node name="Level" type="Node"]
script = ExtResource("lvlid")
{self.solution_metadata()}

[node name="Food" type="Node" parent="."]

[node name="Eaters" type="Node" parent="."]
"""
        for x in range(self.rows):
            for y in range(self.cols):
                cell = self.grid[x][y]
                pos_x = start_x + y*step
                pos_y = start_y + x*step
                
                if isinstance(cell.ent, Food):
                    scene += f"""
[node name="Food{x}_{y}" parent="Food" instance=ExtResource("foodid")]
position = Vector2({pos_x}, {pos_y})
BoardStatePositionId = Vector2i({x}, {y})
FoodType = {self.COLOR_MAP['Food'][cell.ent.color]}
IsLast = {"true" if cell.ent.is_last else "false"}
"""
                elif isinstance(cell.ent, Eater):
                    scene += f"""
[node name="Eater{x}_{y}" parent="Eaters" instance=ExtResource("eaterid")]
position = Vector2({pos_x}, {pos_y})
EaterType = {self.COLOR_MAP['Eater'][cell.ent.color]}
BoardStatePositionId = Vector2i({x}, {y})
ValidFoodTypes = [0, {self.COLOR_MAP['Food'][cell.ent.color]}]
"""
                elif cell.ent is None and fill_empty:
                    scene += f"""
[node name="Food{x}_{y}" parent="Food" instance=ExtResource("foodid")]
position = Vector2({pos_x}, {pos_y})
BoardStatePositionId = Vector2i({x}, {y})
FoodType = 0
IsLast = false
"""
        return scene

    def solution_metadata(self):
        metadata = "metadata/solution = PackedVector2Array("
        for move in self.moves:
                metadata += f"{move["start"][0]}, {move["start"][1]}, {move["end"][0]}, {move["end"][1]}, "
        metadata = metadata[:-2] + ")"
        return metadata

def gen_boards(num_select, total_gen, rows, cols, eaters, white_pct, min_food, weights=None):
    boards = []
    attempts = 0
    max_tries = total_gen * 100
    minimum_qualified_colors = 0 if len(eaters) == 1 else len(eaters)
    max_empty_cells = 1 if len(eaters) == 1 else (int)(rows * cols * 0.1)
    
    # Use a set to track board signatures for similarity detection
    board_signatures = set()
    
    # Add food_balance weight if not present
    if weights is None:
        weights = Grid.DEFAULT_WEIGHTS.copy()
    if 'food_balance' not in weights:
        weights['food_balance'] = 1.8
    
    print("Current attempt: ")
    while len(boards) < total_gen and attempts < max_tries:
        print(f"Current attempt: {attempts}", end="\r")
        g = Grid(rows, cols, weights)
        g.gen_puzzle(eaters, white_pct, min_food)
        
        empty_count = g.count_empty()
        
        if empty_count == 0 and all(
            sum(1 for row in g.grid for cell in row 
                if isinstance(cell.ent, Food) and cell.ent.color == color) >= min_food 
            for color in eaters):
            
            # Get board signature for similarity check
            board_signature = g.get_board_signature()
            
            # Skip if we've already seen a similar board
            if board_signature in board_signatures:
                attempts += 1
                continue
                
            # Add signature to our set
            board_signatures.add(board_signature)
            
            clustering_info = g.calculate_clustering_score(minimum_qualified_colors)
            
            # Skip disqualified boards
            if clustering_info['disqualified'] or empty_count >= max_empty_cells:
                attempts += 1
                continue

            # The final_score from calculate_clustering_score already includes all the weighted factors
            # so we don't need to add them again
            total_score = clustering_info['final_score']
            
            board_info = {
                'board': g,
                'moves': g.moves.copy(),
                'empty': empty_count,
                'clustering_score': clustering_info['final_score'],
                'metrics': clustering_info['color_metrics'],
                'interaction_complexity': clustering_info.get('interaction_complexity', 0),
                'empty_cells_score': clustering_info.get('empty_cells_score', 0),
                'solution_complexity': clustering_info.get('solution_complexity', 0),
                'move_interdependency': clustering_info.get('move_interdependency', 0),
                'critical_path_score': clustering_info.get('critical_path_score', 0),
                'food_balance_score': g.calculate_food_balance_score(),
                'total_score': total_score,
                'signature': board_signature
            }
            
            boards.append(board_info)
        attempts += 1
    
    # Sort by total score, with optional filtering for uniqueness
    boards.sort(key=lambda x: x['total_score'])
    
    return boards[:num_select]


# Example usage
if __name__ == "__main__":
    DIFFICULTY = {
        # ROWS, COLS, WHITE%, MIN_FOOD
        'EASY I': ({'Blue': 1, 'Green': 1}, 4, 4, 10, 2),
        'EASY II': ({'Blue': 1, 'Green': 1}, 4, 4, 12, 2),
        'EASY III': ({'Blue': 1, 'Green': 1}, 5, 4, 15, 2),
        'EASY IV': ({'Blue': 1, 'Green': 1}, 5, 4, 17, 2),
        'EASY V': ({'Blue': 1, 'Green': 1}, 5, 4, 20, 2),
        'MEDIUM I': ({'Blue': 1, 'Green': 1, 'Yellow': 1}, 5, 5, 15, 3),
        'MEDIUM II': ({'Blue': 1, 'Green': 1, 'Yellow': 1}, 5, 5, 17, 3),
        'MEDIUM III': ({'Blue': 1, 'Green': 1, 'Yellow': 1}, 5, 5, 20, 3),
        'MEDIUM IV': ({'Blue': 1, 'Green': 1, 'Yellow': 1}, 6, 5, 23, 3),
        'MEDIUM V': ({'Blue': 1, 'Green': 1, 'Yellow': 1}, 6, 5, 25, 3),
        'HARD I': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1}, 6, 6,  15, 3),
        'HARD II': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1}, 6, 6, 17, 3),
        'HARD III': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1}, 7, 6, 20, 3),
        'HARD IV': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1}, 7, 6, 23, 3),
        'HARD V': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1}, 7, 7, 25, 3),
        'EXPERT I': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1}, 7, 7, 20, 4),
        'EXPERT II': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1}, 7, 7, 23, 4),
        'EXPERT III': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1}, 8, 7, 25, 4),
        'EXPERT IV': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1}, 8, 7, 27, 4),
        'EXPERT V': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1}, 8, 7, 30, 4),
        'GENIUS I': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1, 'Purple': 1}, 9, 7, 25, 5),
        'GENIUS II': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1, 'Purple': 1}, 9, 7, 27, 5),
        'GENIUS III': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1, 'Purple': 1}, 9, 7, 30, 5),
        'GENIUS IV': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1, 'Purple': 1}, 9, 7, 32, 5),
        'GENIUS V': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1, 'Purple': 1}, 9, 7, 35, 5),
        'SUPER I': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1, 'Purple': 1}, 10, 7, 20, 5),
        'SUPER II': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1, 'Purple': 1}, 10, 7, 22, 5),
        'SUPER III': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1, 'Purple': 1}, 10, 7, 25, 5),
        'SUPER IV': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1, 'Purple': 1}, 10, 7, 27, 5),
        'SUPER V': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1, 'Purple': 1}, 11, 7, 30, 5),
        'MASTER I': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1, 'Purple': 1, 'Brown': 1}, 11, 7, 25, 5),
        'MASTER II': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1, 'Purple': 1, 'Brown': 1}, 11, 7, 27, 5),
        'MASTER III': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1, 'Purple': 1, 'Brown': 1}, 11, 7, 30, 5),
        'MASTER IV': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1, 'Purple': 1, 'Brown': 1}, 11, 7, 32, 5),
        'MASTER V': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1, 'Purple': 1, 'Brown': 1}, 11, 7, 35, 5),
    }


    CUSTOM_WEIGHTS = {
        'food_separation': 3,
        'cluster_size': 1,
        'clusters_count': 1,
        'distribution': 3,
        'distance_to_eater': 2,
        'path_complexity': 2,
        'interaction_complexity': 4,
        'empty_cells': 0,
        'solution_complexity': 2,
        'move_interdependency': 4,
        'critical_path': 2,
        'food_balance': 1,
    }

    lvlid = 1
    ts = datetime.datetime.now()
    print(f"[{ts}] Starting")
    for diff, (eaters, rows, cols, white_pct, min_food) in DIFFICULTY.items():
        start_time = datetime.datetime.now()
        num_select = 15
        new_boards = []
        while (len(new_boards) < num_select):
            new_boards += gen_boards(num_select-len(new_boards), 3000, rows, cols, eaters, white_pct, min_food, CUSTOM_WEIGHTS)
        end_time = datetime.datetime.now()
        
        avg_clustering_score = sum([board["clustering_score"] for board in new_boards])/len(new_boards)
        avg_interaction = sum([board.get("interaction_complexity", 0) for board in new_boards])/len(new_boards)
        avg_empty_cells = sum([board.get("empty", 0) for board in new_boards])/len(new_boards)
        avg_solution_complexity = sum([board.get("solution_complexity", 0) for board in new_boards])/len(new_boards)
        avg_move_interdependency = sum([board.get("move_interdependency", 0) for board in new_boards])/len(new_boards)
        avg_critical_path = sum([board.get("critical_path_score", 0) for board in new_boards])/len(new_boards)
        avg_total_score = sum([board.get("total_score", 0) for board in new_boards])/len(new_boards)
        
        print(f"[{datetime.datetime.now()}] Generated {len(new_boards)} new boards for difficulty {diff} ({lvlid}-{lvlid+len(new_boards)-1})")
        print(f"  Avg total score: {avg_total_score:2f}\t Avg clustering score: {avg_clustering_score}")
        print(f"  Avg interaction: {avg_interaction:.2f}\t Avg empty cells: {avg_empty_cells:2f}")
        print(f"  Avg solution complexity: {avg_solution_complexity:.2f}\t Avg move interdependency: {avg_move_interdependency:.2f}")
        print(f"  Avg critical path: {avg_critical_path:.2f}")
        print(f"  Generation time: {end_time - start_time} seconds")
        
        for board in new_boards:
            with open(f"../Levels/Level{lvlid}.tscn", "w") as f:
                f.write(board["board"].to_scene(lvlid, False))
            lvlid += 1

[2025-03-30 16:02:38.973856] Starting
Current attempt: 
[2025-03-30 16:02:38.974858] Generated 1 new boards for difficulty TUTORIAL (0-0)
  Avg total score: inf	 Avg clustering score: inf
  Avg interaction: 0.00	 Avg empty cells: 0.000000
  Avg solution complexity: 20.00	 Avg move interdependency: 14.00
  Avg critical path: 37.50
  Generation time: 0:00:00.001002 seconds
